In [1]:
import requests
import bs4
from bs4 import BeautifulSoup
import pandas as pd
import time
from random import randint

In [2]:
import os,sys,inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0,parentdir) 

from class_definitions import KCW_product

In [3]:
df = pd.read_csv('../kings_county_wines.csv',index_col=0)

In [4]:
wines = list(df['name'])

In [5]:
failed_products = set()
import csv

def getWine_info(KCW_products):
    import time

    #check existing wines
    filename = "../wine-searcher_data.csv"
    reader = csv.DictReader(open(filename))
    already_added = [row for row in reader]
    keys = reader.fieldnames
    
    count = 0
    wine_dicts = []
    for product in KCW_products:
        
        #don't bother scraping dupes
        if product.name in [x['name'] for x in already_added]:
            continue 
        url = product.wine_searcherURL()

        headers = {
            'User-Agent': """Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.130 Safari/537.36""",
            'Connection': 'keep-alive',
            'Accept-Language': 'en-US,en;q=0.9',
            "Accept-Encoding": "gzip, deflate, br",
            "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9",
            'Upgrade-Insecure-Requests': '1',
            'cookie': 'cookie_enabled=true',
            #'referer': f'{wine_searcher_findURL}',
            'sec-fetch-mode': 'navigate',
            'sec-fetch-site': 'same-origin',
            'cache-control': 'max-age=0',
            "sec-fetch-user": "?1"


        }

        page = requests.get(url,headers=headers)
        num_403_errors = 0
        if page.status_code == 403:
            print('403 encountered. Waiting 3 min before resuming')
            num_403_errors +=1
            if num_403_errors>10:
                print("too many 403s! stop what you're doing.")
                break
            # add this wine to the end of the list
            KCW_products.append(product)
            time.sleep(180)
            continue 
        bs = BeautifulSoup(page.content,'lxml')
    
        wine_info_panel = bs.find('div',attrs={"class":'wine-info-panel'})
        if wine_info_panel==None:
            failed_products.add(product)
            continue
        wine_info = []
        
        for div in wine_info_panel.find_all('div',attrs={'class':'dtc'}):
            wine_info.append([x.getText() for x in div.find_all('span',attrs={'class':"dtlbl sidepanel-text"}) if type(x)==bs4.element.Tag])

        wine_info = [item.split('\n') for sublist in wine_info 
                     for item in sublist]
        wine_info = [item.strip() for sublist in wine_info 
                     for item in sublist if item not in  ['',' ']]

        wine_dict = {'name':product.name}
        wine_dict_keys = ['Producer','Region/Appellation','Grape/Blend','Food Suggestion','Wine Style','Alcohol Content','Notes']

        for key in wine_dict_keys:
            try:
                key_index = wine_info.index(key)
                wine_dict[key]=wine_info[key_index+1]
            except:
                wine_dict[key]='null'
        
        # add the new dict to our current data stored in memory
        already_added.append(wine_dict)
        count+=1
        time.sleep(65)
        
    current_data = already_added # data has been updated now
    
    # write the appended data to a new, updated csv 
    f = open(filename,'w')
    writer = csv.DictWriter(f,fieldnames=current_data[0].keys())
    writer.writeheader()
    writer.writerows(current_data)

    return f"{count} records added out of {len(KCW_products)} possible matches"

In [6]:
def chunk(n,list_to_chunk):
    """
    takes in n, and a list to chunk. returns a list of lists with n length. The last chunk size may or may not 
    be equal to n. 
    """
    return [ list_to_chunk[i:i+n] for i in range(0,len(list_to_chunk),n) ]

In [7]:
products = [KCW_product(wine) for wine in wines]

In [8]:
filename = "../wine-searcher_data.csv"
reader = csv.DictReader(open(filename))
already_added = [row['name'] for row in reader]
products_to_add = [product for product in products if product.name not in already_added]


In [17]:
products[0].name

'Mâcon-Loche 2018 Clos des Rocs'

In [9]:
#time.sleep(180)
count = 1
wine_chunks = chunk(4,products_to_add)
for wine_chunk in wine_chunks:
    print(f"starting chunk {count} out of {len(wine_chunks)}")
    getWine_info(wine_chunk)
    time.sleep(60)
    count+=1
    

starting chunk 1 out of 16
starting chunk 2 out of 16
starting chunk 3 out of 16


KeyboardInterrupt: 